In [2]:
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 2000
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128,
                           input_length=max_len,
                           name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed (Embedding)           (None, 500, 128)          256000    
                                                                 
 conv1d (Conv1D)             (None, 494, 32)           28704     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 98, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 92, 32)            7200      
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1)                 3

2022-03-22 10:34:50.501830: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='my_log_dir',
        histogram_freq=1,
        embeddings_freq=1,
    )
]
history = model.fit(x_train, y_train,
                    epochs=5,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=callbacks)

Epoch 1/5
157/157 [==============================] - 66s 416ms/step - loss: 0.5126 - acc: 0.7816 - val_loss: 0.4240 - val_acc: 0.8516
Epoch 2/5
157/157 [==============================] - 58s 367ms/step - loss: 0.4493 - acc: 0.8597 - val_loss: 0.5341 - val_acc: 0.8222
Epoch 3/5
157/157 [==============================] - 58s 369ms/step - loss: 0.3784 - acc: 0.8944 - val_loss: 0.5457 - val_acc: 0.8572
Epoch 4/5
157/157 [==============================] - 51s 326ms/step - loss: 0.3168 - acc: 0.9120 - val_loss: 0.5735 - val_acc: 0.8578
Epoch 5/5
157/157 [==============================] - 61s 388ms/step - loss: 0.2769 - acc: 0.9298 - val_loss: 0.6360 - val_acc: 0.8616


# Advanced Architecture

### Residual Connections
Re-inject data from earlier in the pipeline at a later stage.

### Batch Normalization
Make different samples more similar to each other and easier to learn/generalize.

Basic method:
1. Center on 0 by subtracting mean
2. Scale units by dividing by stdev

Ideally, data should be normalized after each *layer*. The 
`BatchNormalization()` layer does this, and keeps an 
exponential moving average of the batch-wise mean and variance
seen during training. It takes an `axis` argument, which by
default is -1.
```
conv_model.add(layers.Conv2D(32, 3, activation='relu'))
conv_model.add(layers.BatchNormalization())

conv_model.add(layers.Dense(32, activation='relu'))
conv_model.add(layers.BatchNormalization())
```

### Depthwise Separable Convolution
Break input into channels and build convolutions *by channel*.
Smaller, faster and often higher-performing than regular convnet.
```
model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.MaxPooling2D())
```

# Hyperparameter Optimization
